In [1]:
from flash import FLASHQuadConfig, FLASHQuadModel
import torch

In [8]:
config = FLASHQuadConfig()
model = FLASHQuadModel(config)
model.eval()
input_ids = torch.randint(0, 12000, (4, 128))
with torch.no_grad():
    outputs = model(
        input_ids=input_ids, output_attentions=True, output_hidden_states=True
    )
    print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-9.4036e-01,  2.7427e-01, -1.1174e-01,  ...,  2.1995e-01,
           6.8563e-01,  3.7119e-01],
         [ 1.4871e-01,  7.9682e-01,  6.6237e-01,  ...,  1.5242e+00,
           4.3464e-01,  4.5139e-01],
         [ 8.4712e-01,  1.1916e+00,  1.4192e+00,  ...,  5.9129e-01,
           1.6662e-01,  1.2947e+00],
         ...,
         [-1.4811e+00, -4.6251e-01, -1.4264e+00,  ...,  6.2525e-01,
          -2.0381e-01,  3.5314e-01],
         [ 5.8889e-02,  3.1540e-01,  5.3227e-01,  ...,  9.8981e-01,
           1.9056e-01, -1.0363e-01],
         [ 7.3799e-01, -7.2342e-01,  9.3099e-01,  ...,  1.3865e+00,
           2.7682e-02, -4.0529e-01]],

        [[-1.2761e+00,  1.0622e-01,  1.6675e-01,  ...,  6.4745e-01,
           8.1417e-01,  1.8487e-01],
         [ 1.3764e+00,  1.9084e-01,  8.4460e-01,  ...,  1.2662e+00,
          -1.0687e-01,  4.4796e-01],
         [ 5.0415e-01,  1.1916e+00,  8.9216e-01,  ...,  9.3314e-01,
          -1.4971e-01,  3.5086e-01],
     